# Notebook 2: Test Features (No API)

This notebook lets you test the core features directly, without running the API server.

**You'll learn**: How Excel files are flattened, normalized, and compared.

## Setup: Import Everything We Need

In [ ]:
# Add project to path
import sys
from pathlib import Path

project_root = Path.cwd().parent if Path.cwd().name == 'snippets' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Working from: {project_root}")

## Part 1: Create a Test Excel File

Let's create a simple Excel file to work with.

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.comments import Comment

# Create a new workbook
wb = Workbook()
ws = wb.active
ws.title = "Sales Data"

# Add headers with formatting
headers = ['Product', 'Price', 'Quantity', 'Total']
for col, header in enumerate(headers, 1):
    cell = ws.cell(row=1, column=col, value=header)
    cell.font = Font(bold=True, color="FFFFFF")
    cell.fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")

# Add some data
ws['A2'] = 'Widget'
ws['B2'] = 10.50
ws['C2'] = 5
ws['D2'] = '=B2*C2'  # Formula!

ws['A3'] = 'Gadget'
ws['B3'] = 25.00
ws['C3'] = 3
ws['D3'] = '=B3*C3'  # Formula!

# Add a total row
ws['A4'] = 'TOTAL'
ws['D4'] = '=SUM(D2:D3)'  # Formula!
ws['D4'].font = Font(bold=True)

# Add a comment
ws['D4'].comment = Comment('This is the total sales', 'Excel Bot')

# Add a second sheet
ws2 = wb.create_sheet("Summary")
ws2['A1'] = 'Summary'
ws2['A2'] = 'Total Sales:'
ws2['B2'] = '=\'Sales Data\'!D4'  # Reference to other sheet!

# Save it
test_file = Path('/workspaces/excel-differ/tmp/test_workbook.xlsx')
wb.save(test_file)

print(f"✅ Created test Excel file: {test_file}")
print(f"   - 2 sheets")
print(f"   - Formulas (SUM, multiplication, cross-sheet reference)")
print(f"   - Formatting (bold, colors)")
print(f"   - Comments")

## Part 2: Test Formula Normalization

See how formulas are normalized to be deterministic.

In [ ]:
from src.engine.flattener.normalizer import normalize_formula

test_formulas = [
    "=sum(A1:A10)",              # lowercase function
    "= SUM(A1:A10) ",            # extra spaces
    "=Sum(A1:A10)",              # mixed case
    "=average(B1:B5)",           # another function
    "=IF(C1>100, \"High\", \"Low\")",  # complex formula
]

print("Formula Normalization Examples:\n")
print("="*70)

for formula in test_formulas:
    normalized = normalize_formula(formula)
    print(f"Input:  {formula:40s}")
    print(f"Output: {normalized:40s}")
    print("-"*70)

## Part 3: Test Number Normalization

Numbers are normalized to 15 significant digits (Excel's precision).

In [ ]:
from src.engine.flattener.normalizer import normalize_number

test_numbers = [
    123,
    123.456,
    123.456789012345678,  # More than 15 digits
    0.00000123,
    1e10,
    "1,234.56",  # String with formatting
]

print("Number Normalization Examples:\n")
print("="*70)

for num in test_numbers:
    normalized = normalize_number(num)
    print(f"Input:  {str(num):25s} ({type(num).__name__})")
    print(f"Output: {normalized:25s}")
    print("-"*70)

## Part 4: Flatten the Test Excel File

**This is the main operation!** Converting Excel → Text files.

In [ ]:
from src.engine.flattener.workbook import WorkbookFlattener

# Create flattener
flattener = WorkbookFlattener(include_evaluated=False)

# Flatten the test file we created
print("Flattening test_workbook.xlsx...\n")

result = flattener.flatten(
    input_file=test_file,
    output_dir=Path('/workspaces/excel-differ/tmp/excel-differ-test')
)

snapshot_dir = result['snapshot_dir']
manifest = result['manifest']

print(f"✅ Flattening complete!\n")
print(f"Snapshot location: {snapshot_dir}\n")
print(f"Manifest info:")
print(f"  Filename: {manifest.workbook_filename}")
print(f"  SHA256: {manifest.original_sha256[:16]}...")
print(f"  Sheets: {len(manifest.sheets)}")

for sheet in manifest.sheets:
    vis = "Visible" if sheet['visible'] else "Hidden"
    print(f"    - {sheet['name']} ({vis})")

if result['warnings']:
    print(f"\n⚠️ Warnings: {result['warnings']}")

## Part 5: Explore the Flattened Output

Let's see what files were created!

In [ ]:
import os

print(f"Files created in snapshot:\n")
print("="*70)

for root, dirs, files in os.walk(snapshot_dir):
    level = root.replace(str(snapshot_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    folder_name = os.path.basename(root) or snapshot_dir.name
    print(f"{indent}📁 {folder_name}/")
    
    subindent = ' ' * 2 * (level + 1)
    for file in sorted(files):
        file_path = Path(root) / file
        size = file_path.stat().st_size
        print(f"{subindent}📄 {file} ({size} bytes)")

## Part 6: Look at Extracted Formulas

See what the formula file looks like.

In [ ]:
# Find the formulas file for first sheet
formulas_file = list(snapshot_dir.glob("sheets/*formulas.txt"))[0]

print(f"Contents of: {formulas_file.name}\n")
print("="*70)

with open(formulas_file, 'r') as f:
    content = f.read()
    print(content)

print("="*70)
print("\nNotice:")
print("  - Formulas are UPPERCASE (normalized)")
print("  - Cell addresses in column 1")
print("  - Tab-separated")
print("  - Sorted by cell address")

## Part 7: Look at Extracted Values

See what the hardcoded values file looks like.

In [ ]:
# Find the values file
values_file = list(snapshot_dir.glob("sheets/*values_hardcoded.txt"))[0]

print(f"Contents of: {values_file.name}\n")
print("="*70)

with open(values_file, 'r') as f:
    content = f.read()
    print(content)

print("="*70)
print("\nNotice:")
print("  - Only non-formula cells (hard-coded values)")
print("  - Numbers, strings, etc.")
print("  - Same tab-separated format")

## Part 8: Look at the Manifest

The manifest.json contains all metadata.

In [ ]:
import json

manifest_file = snapshot_dir / "manifest.json"

print("Manifest contents:\n")
print("="*70)

with open(manifest_file, 'r') as f:
    manifest_data = json.load(f)
    print(json.dumps(manifest_data, indent=2))

print("\n" + "="*70)
print("The manifest lists:")
print("  - All sheets")
print("  - All files with their SHA256 hashes")
print("  - Extraction metadata")
print("  - Any warnings")

## Part 9: Modify the Excel File

Let's change something and create version 2.

In [ ]:
from openpyxl import load_workbook

# Load the original file
wb = load_workbook(test_file)
ws = wb['Sales Data']

# Make some changes
ws['B2'] = 12.00  # Change price
ws['C2'] = 10     # Change quantity

# Add a new row
ws['A5'] = 'Doodad'
ws['B5'] = 5.00
ws['C5'] = 20
ws['D5'] = '=B5*C5'

# Update total formula
ws['D4'] = '=SUM(D2:D5)'

# Save as version 2
test_file_v2 = Path('/workspaces/excel-differ/tmp/test_workbook_v2.xlsx')
wb.save(test_file_v2)

print(f"✅ Created modified version: {test_file_v2}")
print("\nChanges made:")
print("  - Changed Widget price: 10.50 → 12.00")
print("  - Changed Widget quantity: 5 → 10")
print("  - Added new product: Doodad")
print("  - Updated total formula")

## Part 10: Flatten Version 2

In [ ]:
# Flatten version 2
print("Flattening version 2...\n")

result_v2 = flattener.flatten(
    input_file=test_file_v2,
    output_dir=Path('/workspaces/excel-differ/tmp/excel-differ-test')
)

snapshot_dir_v2 = result_v2['snapshot_dir']

print(f"✅ Version 2 flattened!")
print(f"   V1: {snapshot_dir.name}")
print(f"   V2: {snapshot_dir_v2.name}")

## Part 11: Compare the Two Versions

**This is the diff engine in action!**

In [ ]:
from src.engine.differ.compare import SnapshotComparison
from src.engine.differ.diff_json import generate_json_diff

# Create comparison
print("Comparing V1 and V2...\n")

comparison = SnapshotComparison(snapshot_dir, snapshot_dir_v2)

# Get file-level changes
changes = comparison.get_changed_files()

print("File-level changes:")
print(f"  Added files: {len(changes['added'])}")
print(f"  Removed files: {len(changes['removed'])}")
print(f"  Modified files: {len(changes['modified'])}")

if changes['modified']:
    print(f"\nModified files:")
    for file in changes['modified']:
        print(f"  - {file}")

## Part 12: Generate Structured Diff

Get a detailed, typed diff showing exactly what changed.

In [ ]:
# Generate structured diff
diff_result = generate_json_diff(comparison)

print("Structured Diff Summary:\n")
print("="*70)

for key, count in diff_result['summary'].items():
    if count > 0:
        print(f"{key}: {count}")

print("\n" + "="*70)
print("\nDetailed Changes:\n")

for change in diff_result['diff_json'][:20]:  # Show first 20 changes
    print(json.dumps(change, indent=2))
    print("-"*70)

## ✅ You Did It!

**What you learned:**
1. ✅ How formulas and numbers are normalized
2. ✅ How Excel files are flattened into text files
3. ✅ What the snapshot directory structure looks like
4. ✅ How to compare two snapshots
5. ✅ How to get structured diff output

**Next step**: Open `03_test_api.ipynb` to test the REST API!